In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from geopy.distance import vincenty
import types

In [5]:
reviews_file = "./data/base_DFG_note_feat_pp_loc"
reviews = pd.read_csv(reviews_file + ".csv")
reviews.shape

(9665, 70)

In [6]:
# Add coords column
reviews["coords"] = reviews["latitude"].map(str) + "," + reviews["longitude"].map(str)
reviews.head(5)

,Adresse,Code Postal,Resto,Ville,Note_resto,Note_hygiène_resto,Variance_note_resto,Variance_note_hygiène_resto,16,alimentair,...,traversent,ventr,vom,écrev,épic,éton,rev_cnt,latitude,longitude,coords
0,55 Boulevard Saint Marcel,75013,0 d'attente,Paris,3.900000,3.670000,0.290000,2.269600,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,10.0,48.837656,2.355498,"48.8376561,2.3554979"
1,"128, rue du Faubourg Saint Martin",75010,0039 ristorante italiano,Paris,3.222222,4.846111,2.172840,0.020053,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,48.874388,2.358907,"48.8743883,2.3589067999999997"
2,60 rue Albert,75013,015 gang nam,Paris,4.333333,4.844444,0.222222,0.010617,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,48.826235,2.371878,"48.82623539999999,2.371877500000001"
3,161 Avenue D'Italie,75013,1 pot,Paris,4.000000,4.916667,0.666667,0.000278,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,48.819530,2.359702,"48.8195302,2.3597021000000002"
4,55 Boulevard Saint Marcel,75013,0 d'attente,Paris,3.900000,3.670000,0.290000,2.269600,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,1.0,10.0,48.837656,2.355498,"48.8376561,2.3554979"


In [7]:
# Keep only instances with coordinates
reviews = reviews[(reviews["latitude"] > 0) & (reviews["longitude"] > 0)]
reviews.shape

(9661, 70)

In [8]:
inspections_file =  "./data/inspections-restaurants_pp"
inspections = pd.read_csv(inspections_file + ".csv")

In [9]:
inspections.head(5)
# inspections.dtypes

,Nom,SIRET,Adresse,Code postal,Localité,Date inspection,Note globale,Fermeture,Note,Evaluation,Coordonnées géographiques,Date extraction,latitude,longitude
0,THESTEACKFRITES,7.915752e+13,10 RUE GOMBOUST,75001.0,PARIS 1,2015-06-22,C,0.0,note D,20.0,"48.867387155, 2.332735426",2015-01-07,48.867387,2.332735
1,UMAMI,8.004104e+13,7 RUE DU VINGT NEUF JUILLET,75001.0,PARIS 1,2015-04-28,C,0.0,note D,20.0,"48.863195893, 2.336133862",2015-01-07,48.863196,2.336134
2,S&H,8.032046e+13,9 RUE DES PETITS CHAMPS,75001.0,PARIS 1,2015-04-24,B,0.0,note B,10.0,"48.866272815, 2.338753853",2015-01-07,48.866273,2.338754
3,SOCIETE DE GERANCE DES FRANCS BOURGEOI,8.049760e+13,12 RUE COQUILLIERE,75001.0,PARIS,2015-04-07,C,0.0,note D,20.0,"48.863546941, 2.343246508",2015-01-07,48.863547,2.343247
4,BERTHELOT JEAN MARC,4.084507e+13,22 RUE DES CAPUCINES,75002.0,PARIS 2,2015-05-07,C,0.0,note D,20.0,"48.869444864, 2.328647621",2015-01-07,48.869445,2.328648


In [10]:
# Keep only instances with coordinates
inspections = inspections[~pd.isnull(inspections["Coordonnées géographiques"])]
inspections.shape

(843, 14)

In [11]:
inspections.head(5)

,Nom,SIRET,Adresse,Code postal,Localité,Date inspection,Note globale,Fermeture,Note,Evaluation,Coordonnées géographiques,Date extraction,latitude,longitude
0,THESTEACKFRITES,7.915752e+13,10 RUE GOMBOUST,75001.0,PARIS 1,2015-06-22,C,0.0,note D,20.0,"48.867387155, 2.332735426",2015-01-07,48.867387,2.332735
1,UMAMI,8.004104e+13,7 RUE DU VINGT NEUF JUILLET,75001.0,PARIS 1,2015-04-28,C,0.0,note D,20.0,"48.863195893, 2.336133862",2015-01-07,48.863196,2.336134
2,S&H,8.032046e+13,9 RUE DES PETITS CHAMPS,75001.0,PARIS 1,2015-04-24,B,0.0,note B,10.0,"48.866272815, 2.338753853",2015-01-07,48.866273,2.338754
3,SOCIETE DE GERANCE DES FRANCS BOURGEOI,8.049760e+13,12 RUE COQUILLIERE,75001.0,PARIS,2015-04-07,C,0.0,note D,20.0,"48.863546941, 2.343246508",2015-01-07,48.863547,2.343247
4,BERTHELOT JEAN MARC,4.084507e+13,22 RUE DES CAPUCINES,75002.0,PARIS 2,2015-05-07,C,0.0,note D,20.0,"48.869444864, 2.328647621",2015-01-07,48.869445,2.328648


In [30]:
# Initialize merged DataFrame
merged = pd.DataFrame(columns=list(reviews.columns.values))

In [31]:
idx = 0
# Loop through inspected restaurant
for (nom, coord), grp in inspections.groupby(["Nom", "Coordonnées géographiques"], as_index=False):
    if idx < 2:
        print(nom, coord)
        print(grp["Note globale"])
    inspection_loc = tuple([float(x) for x in coord.split(",")])
    # print(inspection_loc)
    # Loop thorugh reviews
    for index, review_row in reviews.iterrows():
        review_loc = tuple([float(x) for x in review_row["coords"].split(",")])
        # Check name-location match
        if (fuzz.token_sort_ratio(review_row["Resto"], nom) >= 90) & (vincenty(review_loc, inspection_loc).meters < 100):
            #print(type(grp["Note globale"].values[0]))   
            inspection_note = grp["Note globale"].values[0] # TODO This is a workaround, we need to support multiple notes
            merged_entry = review_row.append(pd.Series(inspection_note))
            merged = merged.append(merged_entry, ignore_index=True)
            print("Match!!!\n", review_row["Resto"], nom)
            # print(merged_entry)
    idx += 1
print(idx)

2 B C 48.862613813, 2.27656728
646    C
Name: Note globale, dtype: object
2 DEA 48.840816512, 2.344770086
51    C
Name: Note globale, dtype: object


/home/jacob/fellow/env/lib/python3.5/site-packages/ipykernel/__main__.py:17: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects


Match!!!
 african lounge AFRICAN LOUNGE
Match!!!
 al solito posto AL SOLITO POSTO
Match!!!
 al solito posto AL SOLITO POSTO
Match!!!
 american bistro AMERICAN BISTROT
Match!!!
 asian wok ASIAN WOK
Match!!!
 auto passion café AUTO PASSION CAFE
Match!!!
 bistrot beyrouth BISTROT BEYROUTH
Match!!!
 bistrot osaka BISTROT OSAKA
Match!!!
 bistrot valois BISTROT VALOIS
Match!!!
 bottles BOTTLES
Match!!!
 brasserie de la porte dauphine BRASSERIE DE LA PORTE DAUPHINE
Match!!!
 cafe d aujourd hui CAFE D'AUJOURD'HUI
Match!!!
 cafe de flore CAFE DE FLORE
Match!!!
 cafe du marche des enfants rouges CAFE DU MARCHE DES ENFANTS ROUGES
Match!!!
 casa botticelli CASA BOTTICELLI
Match!!!
 cheval d'or CHEVAL D OR
Match!!!
 chez fernand christine CHEZ FERNAND CHRISTINE
Match!!!
 chez jenny CHEZ JENNY
Match!!!
 chez van CHEZ VAN
Match!!!
 clasico argentino CLASICO ARGENTINO
Match!!!
 le comptoir du faubourg COMPTOIR DU FAUBOURG
Match!!!
 côté sushi marceau COTE SUSHI MARCEAU
Match!!!
 cristal CRISTAL
Match!

In [32]:
# Merged data frame
merged_cols = list(reviews.columns)
merged_cols.append("inspection_note")
print(merged_cols)

merged.columns = merged_cols
merged.head()

['Adresse', 'Code Postal', 'Resto', 'Ville', 'Note_resto', 'Note_hygiène_resto', 'Variance_note_resto', 'Variance_note_hygiène_resto', '16', 'alimentair', 'assaison', 'attend', 'brul', 'canap', 'cher', 'commissariat', 'comprenon', 'courent', 'cri', 'cru', 'cuisin', 'désoblig', 'except', 'fraich', 'gaufr', 'gav', 'glu', 'gu', 'haw', 'horribl', 'indigest', 'infect', 'intox', 'lac', 'lendemain', 'loc', 'malad', 'marn', 'miush', 'nantu', 'nor', 'nuit', 'oblig', 'oreil', 'pert', 'plas', 'poivr', 'pompon', 'potag', 'poubel', 'propos', 'pulper', 'pékinois', 'remarqu', 'retiendr', 'réfrig', 'scandal', 'sept', 'sourd', 'tomb', 'traversent', 'ventr', 'vom', 'écrev', 'épic', 'éton', 'rev_cnt', 'latitude', 'longitude', 'coords', 'inspection_note']


,Adresse,Code Postal,Resto,Ville,Note_resto,Note_hygiène_resto,Variance_note_resto,Variance_note_hygiène_resto,16,alimentair,...,ventr,vom,écrev,épic,éton,rev_cnt,latitude,longitude,coords,inspection_note
0,20 Bis Rue Jean Giraudoux,75116,african lounge,Paris,3.000000,5.000000,2.000000,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,48.869483,2.296996,"48.869483100000004,2.2969957",C
1,4 Place Edmond Michelet,75004,al solito posto,Paris,4.500000,4.883333,0.250000,0.000278,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,48.859915,2.350519,"48.859915,2.3505187000000003",C
2,91 rue saint martin,75004,al solito posto,Paris,4.500000,4.883333,0.250000,0.000278,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,48.859907,2.350298,"48.8599074,2.3502982999999995",C
3,74 Rue de la Folie Mericourt,75011,american bistro,Paris,3.666667,4.950000,1.555556,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,48.865611,2.370803,"48.86561079999999,2.3708026",B
4,"63, Rue Oberkampf",75011,asian wok,Paris,4.000000,4.180556,0.571429,0.696867,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,48.864862,2.373177,"48.8648624,2.373177",C


In [33]:
merged.shape

(113, 71)

In [34]:
merged_file = "./data/restaurants_feat_label.csv"
merged.to_csv(merged_file, index=False, sep=";")

In [35]:
# For plotting purposes only
merged[["latitude", "longitude", "inspection_note"]].to_csv("./data/plot_matches.csv", index=False, sep=";")